### Spark notebook ###

This notebook will only work in a Jupyter session running on `mathmadslinux2p`.

You can start your own Jupyter session on `mathmadslinux2p` and open this notebook in Chrome on the MADS Windows server by

**Steps**

1. Login to the MADS Windows server using https://mathportal.canterbury.ac.nz/.
2. Download or copy this notebook to your home directory.
3. Open powershell and run `ssh mathmadslinux2p`.
4. Run `start_pyspark_notebook` or `/opt/anaconda3/bin/jupyter-notebook --ip 132.181.129.68 --port $((8000 + $((RANDOM % 999))))`.
5. Copy / paste the url provided in the shell window into Chrome on the MADS Windows server.
6. Open the notebook from the Jupyter root directory (which is your home directory).
7. Run `start_spark()` to start a spark session in the notebook.
8. Run `stop_spark()` before closing the notebook or kill your spark application by hand using the link in the Spark UI.

In [11]:
# Run this cell to import pyspark and to define start_spark() and stop_spark()

import findspark

findspark.init()

import getpass
import pandas as pd
import pyspark
import random
import re

from IPython.display import display, HTML
from pyspark import SparkContext
from pyspark.sql import SparkSession


# Functions used below

def username():
    """Get username with any domain information removed.
    """

    return re.sub('@.*', '', getpass.getuser())


def dict_to_html(d):
    """Convert a Python dictionary into a two column table for display.
    """

    html = []

    html.append(f'<table width="100%" style="width:100%; font-family: monospace;">')
    for k, v in d.items():
        html.append(f'<tr><td style="text-align:left;">{k}</td><td>{v}</td></tr>')
    html.append(f'</table>')

    return ''.join(html)


def show_as_html(df, n=10):
    """Leverage existing pandas jupyter integration to show a spark dataframe as html.
    
    Args:
        n (int): number of rows to show (default: 20)
    """

    display(df.limit(n).toPandas())

    
def display_spark():
    """Display the status of the active Spark session if one is currently running.
    """
    
    if 'spark' in globals() and 'sc' in globals():

        name = sc.getConf().get("spark.app.name")
        
        html = [
            f'<p><b>Spark</b></p>',
            f'<p>The spark session is <b><span style="color:green">active</span></b>, look for <code>{name}</code> under the running applications section in the Spark UI.</p>',
            f'<ul>',
            f'<li><a href="http://mathmadslinux2p.canterbury.ac.nz:8080/" target="_blank">Spark UI</a></li>',
            f'<li><a href="{sc.uiWebUrl}" target="_blank">Spark Application UI</a></li>',
            f'</ul>',
            f'<p><b>Config</b></p>',
            dict_to_html(dict(sc.getConf().getAll())),
            f'<p><b>Notes</b></p>',
            f'<ul>',
            f'<li>The spark session <code>spark</code> and spark context <code>sc</code> global variables have been defined by <code>start_spark()</code>.</li>',
            f'<li>Please run <code>stop_spark()</code> before closing the notebook or restarting the kernel or kill <code>{name}</code> by hand using the link in the Spark UI.</li>',
            f'</ul>',
        ]
        display(HTML(''.join(html)))
        
    else:
        
        html = [
            f'<p><b>Spark</b></p>',
            f'<p>The spark session is <b><span style="color:red">stopped</span></b>, confirm that <code>{username() + " (jupyter)"}</code> is under the completed applications section in the Spark UI.</p>',
            f'<ul>',
            f'<li><a href="http://mathmadslinux2p.canterbury.ac.nz:8080/" target="_blank">Spark UI</a></li>',
            f'</ul>',
        ]
        display(HTML(''.join(html)))


# Functions to start and stop spark

def start_spark(executor_instances=2, executor_cores=1, worker_memory=1, master_memory=1):
    """Start a new Spark session and define globals for SparkSession (spark) and SparkContext (sc).
    
    Args:
        executor_instances (int): number of executors (default: 2)
        executor_cores (int): number of cores per executor (default: 1)
        worker_memory (float): worker memory (default: 1)
        master_memory (float): master memory (default: 1)
    """

    global spark
    global sc

    user = username()
    
    cores = executor_instances * executor_cores
    partitions = cores * 4
    port = 4000 + random.randint(1, 999)

    spark = (
        SparkSession.builder
        .master("spark://masternode2:7077")
        .config("spark.driver.extraJavaOptions", f"-Dderby.system.home=/tmp/{user}/spark/")
        .config("spark.dynamicAllocation.enabled", "false")
        .config("spark.executor.instances", str(executor_instances))
        .config("spark.executor.cores", str(executor_cores))
        .config("spark.cores.max", str(cores))
        .config("spark.executor.memory", f"{worker_memory}g")
        .config("spark.driver.memory", f"{master_memory}g")
        .config("spark.driver.maxResultSize", "0")
        .config("spark.sql.shuffle.partitions", str(partitions))
        .config("spark.ui.port", str(port))
        .appName(user + " (jupyter)")
        .getOrCreate()
    )
    sc = SparkContext.getOrCreate()
    
    display_spark()

    
def stop_spark():
    """Stop the active Spark session and delete globals for SparkSession (spark) and SparkContext (sc).
    """

    global spark
    global sc

    if 'spark' in globals() and 'sc' in globals():

        spark.stop()

        del spark
        del sc

    display_spark()


# Make css changes to improve spark output readability

html = [
    '<style>',
    'pre { white-space: pre !important; }',
    'table.dataframe td { white-space: nowrap !important; }',
    'table.dataframe thead th:first-child, table.dataframe tbody th { display: none; }',
    '</style>',
]
display(HTML(''.join(html)))

In [12]:
# Run this cell to start a spark session in this notebook

start_spark(executor_instances=4, executor_cores=2, worker_memory=4, master_memory=4)

spark.app.name,kda115 (jupyter)
spark.dynamicAllocation.enabled,false
spark.executor.instances,4
spark.driver.port,39021
spark.driver.memory,4g
spark.executor.memory,4g
spark.master,spark://masternode2:7077
spark.executor.id,driver
spark.app.id,app-20241021191417-0120
spark.ui.port,4088
spark.executor.cores,2


## Question 02 

In [13]:
# Write your imports and code here or insert cells below

from pyspark.sql import Row, DataFrame, Window, functions as F
from pyspark.sql.types import *

### A. Load the audio feature attribute names and types from the audio/attributes directory 

The Attribute DataFrame contain columns like of Attribute Name and AttributeType, which describe the feature names and the corresponding types

In [14]:
# Create a dictionary for mapping attribute types
dict_type = {
    'float': DoubleType(),
    'string': StringType(),
    'STRING': StringType(),
    'NUMERIC': DoubleType(),
    'real': DoubleType()
}

In [15]:
# Pick one dataset to create a schema 
name = 'msd-jmir-area-of-moments-all-v1.0'

In [16]:
# Define a function to create a schema from a DataFrame containing attribute names and types
def generate_schema(attribute_df):
    '''Generate a schema based on attribute information about feature names and types'''

    # Initialize an empty list to hold StructField objects
    struct_fields = []

    # Loop through each row in the DataFrame
    for attribute_info in attribute_df.collect():
        
        # Fetch the data type from the type_dict mapping based on the attribute type
        data_type = dict_type[attribute_info['type']]
        
        # Create a StructField for each attribute name and its corresponding type
        field = StructField(attribute_info['name'], data_type, True)
        
        # Append the StructField to the list
        struct_fields.append(field)
        
    # Return a StructType created from the list of StructField objects
    return StructType(struct_fields)

#### A.1  Create schema for feature and load feature dataset

In [17]:
# Set directory path containing attribute and feature files
attribute_path = f'hdfs:///data/msd/audio/attributes/{name}.attributes.csv'
feature_path = f'hdfs:///data/msd/audio/features/{name}.csv'
    
# Load 'attribute' dataset  
attribute_data = (spark.read.format('csv')
                  .option('header', False)
                  .load(attribute_path)
                  .toDF('name', 'type'))

# Create a schema for feature using attribute name and type
feature_schema = generate_schema(attribute_data)

# show the result 
feature_schema

StructType(List(StructField(Area_Method_of_Moments_Overall_Standard_Deviation_1,DoubleType,true),StructField(Area_Method_of_Moments_Overall_Standard_Deviation_2,DoubleType,true),StructField(Area_Method_of_Moments_Overall_Standard_Deviation_3,DoubleType,true),StructField(Area_Method_of_Moments_Overall_Standard_Deviation_4,DoubleType,true),StructField(Area_Method_of_Moments_Overall_Standard_Deviation_5,DoubleType,true),StructField(Area_Method_of_Moments_Overall_Standard_Deviation_6,DoubleType,true),StructField(Area_Method_of_Moments_Overall_Standard_Deviation_7,DoubleType,true),StructField(Area_Method_of_Moments_Overall_Standard_Deviation_8,DoubleType,true),StructField(Area_Method_of_Moments_Overall_Standard_Deviation_9,DoubleType,true),StructField(Area_Method_of_Moments_Overall_Standard_Deviation_10,DoubleType,true),StructField(Area_Method_of_Moments_Overall_Average_1,DoubleType,true),StructField(Area_Method_of_Moments_Overall_Average_2,DoubleType,true),StructField(Area_Method_of_Moment

In [18]:
# Load 'feature' dataset  
feature_data = (spark.read.format('csv')
                .option('header', False)
                .option('inferSchema', False)
                .option('quote', "'")  # Ensures quotes ' ' in CSV are treated as delimiters, not data
                .schema(feature_schema)
                .load(feature_path))
    
    
# Show the result
print(f'Feature data corresponding to {name} dataset:')
show_as_html(feature_data, 5)

Feature data corresponding to msd-jmir-area-of-moments-all-v1.0 dataset:


,Area_Method_of_Moments_Overall_Standard_Deviation_1,Area_Method_of_Moments_Overall_Standard_Deviation_2,Area_Method_of_Moments_Overall_Standard_Deviation_3,Area_Method_of_Moments_Overall_Standard_Deviation_4,Area_Method_of_Moments_Overall_Standard_Deviation_5,Area_Method_of_Moments_Overall_Standard_Deviation_6,Area_Method_of_Moments_Overall_Standard_Deviation_7,Area_Method_of_Moments_Overall_Standard_Deviation_8,Area_Method_of_Moments_Overall_Standard_Deviation_9,Area_Method_of_Moments_Overall_Standard_Deviation_10,...,Area_Method_of_Moments_Overall_Average_2,Area_Method_of_Moments_Overall_Average_3,Area_Method_of_Moments_Overall_Average_4,Area_Method_of_Moments_Overall_Average_5,Area_Method_of_Moments_Overall_Average_6,Area_Method_of_Moments_Overall_Average_7,Area_Method_of_Moments_Overall_Average_8,Area_Method_of_Moments_Overall_Average_9,Area_Method_of_Moments_Overall_Average_10,MSD_TRACKID
0,1.2000,3355.0,26270.0,39850000.0,3.096000e+08,2.403000e+09,8.749000e+11,2.775000e+09,2.155000e+10,4.064000e+14,...,5746.0,43470.0,-44220000.0,-3.376000e+08,-2.576000e+09,7.665000e+11,3.015000e+09,2.302000e+10,3.460000e+14,TRHFHQZ12903C9E2D5
1,0.9295,6720.0,44100.0,160800000.0,1.060000e+09,6.985000e+09,7.095000e+12,9.545000e+09,6.293000e+10,2.037000e+15,...,11580.0,74040.0,-179200000.0,-1.153000e+09,-7.420000e+09,6.242000e+12,1.037000e+10,6.680000e+10,1.694000e+15,TRHFHYX12903CAF953
2,1.8830,6712.0,49060.0,160600000.0,1.176000e+09,8.609000e+09,7.083000e+12,1.058000e+10,7.744000e+10,2.781000e+15,...,11580.0,85200.0,-179100000.0,-1.316000e+09,-9.660000e+09,6.233000e+12,1.182000e+10,8.680000e+10,2.463000e+15,TRHFHAU128F9341A0E
3,1.8840,6722.0,56130.0,161000000.0,1.346000e+09,1.127000e+10,7.112000e+12,1.211000e+10,1.014000e+11,4.193000e+15,...,11600.0,93320.0,-179700000.0,-1.459000e+09,-1.185000e+10,6.262000e+12,1.311000e+10,1.066000e+11,3.432000e+15,TRHFHLP128F14947A7
4,1.5200,6709.0,53230.0,160500000.0,1.295000e+09,1.045000e+10,7.076000e+12,1.164000e+10,9.392000e+10,3.751000e+15,...,11580.0,93650.0,-179000000.0,-1.441000e+09,-1.159000e+10,6.230000e+12,1.293000e+10,1.041000e+11,3.248000e+15,TRHFHFF128F930AC11


### B. Load more of the audio feature attribute datasets from the audio/features directory using the generated schema 

In [15]:
# List of dataset names
dataset_names = [
    'msd-jmir-area-of-moments-all-v1.0',
    'msd-jmir-lpc-all-v1.0', 
    'msd-jmir-methods-of-moments-all-v1.0', 
    'msd-jmir-mfcc-all-v1.0',
    'msd-jmir-spectral-all-all-v1.0',
    'msd-jmir-spectral-derivatives-all-all-v1.0',
    'msd-marsyas-timbral-v1.0',
    'msd-mvd-v1.0',
    'msd-rh-v1.0',
    'msd-rp-v1.0',
    'msd-ssd-v1.0',
    'msd-trh-v1.0',
    'msd-tssd-v1.0',   
]

# Loop through each dataset name
for dataset_name in dataset_names:
    
    # Set paths dynamically for the attribute and feature files
    attribute_path = f'hdfs:///data/msd/audio/attributes/{dataset_name}.attributes.csv'
    feature_path = f'hdfs:///data/msd/audio/features/{dataset_name}.csv'
    
    # Load attribute data to create the schema
    attribute_data = (spark.read.format('csv')
                      .option('header', False)
                      .load(attribute_path)
                      .toDF('name', 'type'))  # Ensure the column names are consistent
    
    # Create schema using the attribute data
    feature_schema = generate_schema(attribute_data)
    
    # Load feature data using the generated schema
    feature_data = (spark.read.format('csv')
                    .option('header', False)
                    .option('inferSchema', False)
                    .option('quote', '')  # Ensures quotes in CSV are handled correctly
                    .schema(feature_schema)
                    .load(feature_path))
    
    # Show the result for each dataset
    print(f'Feature data corresponding to dataset name: {dataset_name}:')
    show_as_html(feature_data, 5)

Feature data corresponding to dataset name: msd-jmir-area-of-moments-all-v1.0:


,Area_Method_of_Moments_Overall_Standard_Deviation_1,Area_Method_of_Moments_Overall_Standard_Deviation_2,Area_Method_of_Moments_Overall_Standard_Deviation_3,Area_Method_of_Moments_Overall_Standard_Deviation_4,Area_Method_of_Moments_Overall_Standard_Deviation_5,Area_Method_of_Moments_Overall_Standard_Deviation_6,Area_Method_of_Moments_Overall_Standard_Deviation_7,Area_Method_of_Moments_Overall_Standard_Deviation_8,Area_Method_of_Moments_Overall_Standard_Deviation_9,Area_Method_of_Moments_Overall_Standard_Deviation_10,...,Area_Method_of_Moments_Overall_Average_2,Area_Method_of_Moments_Overall_Average_3,Area_Method_of_Moments_Overall_Average_4,Area_Method_of_Moments_Overall_Average_5,Area_Method_of_Moments_Overall_Average_6,Area_Method_of_Moments_Overall_Average_7,Area_Method_of_Moments_Overall_Average_8,Area_Method_of_Moments_Overall_Average_9,Area_Method_of_Moments_Overall_Average_10,MSD_TRACKID
0,1.2000,3355.0,26270.0,39850000.0,3.096000e+08,2.403000e+09,8.749000e+11,2.775000e+09,2.155000e+10,4.064000e+14,...,5746.0,43470.0,-44220000.0,-3.376000e+08,-2.576000e+09,7.665000e+11,3.015000e+09,2.302000e+10,3.460000e+14,'TRHFHQZ12903C9E2D5'
1,0.9295,6720.0,44100.0,160800000.0,1.060000e+09,6.985000e+09,7.095000e+12,9.545000e+09,6.293000e+10,2.037000e+15,...,11580.0,74040.0,-179200000.0,-1.153000e+09,-7.420000e+09,6.242000e+12,1.037000e+10,6.680000e+10,1.694000e+15,'TRHFHYX12903CAF953'
2,1.8830,6712.0,49060.0,160600000.0,1.176000e+09,8.609000e+09,7.083000e+12,1.058000e+10,7.744000e+10,2.781000e+15,...,11580.0,85200.0,-179100000.0,-1.316000e+09,-9.660000e+09,6.233000e+12,1.182000e+10,8.680000e+10,2.463000e+15,'TRHFHAU128F9341A0E'
3,1.8840,6722.0,56130.0,161000000.0,1.346000e+09,1.127000e+10,7.112000e+12,1.211000e+10,1.014000e+11,4.193000e+15,...,11600.0,93320.0,-179700000.0,-1.459000e+09,-1.185000e+10,6.262000e+12,1.311000e+10,1.066000e+11,3.432000e+15,'TRHFHLP128F14947A7'
4,1.5200,6709.0,53230.0,160500000.0,1.295000e+09,1.045000e+10,7.076000e+12,1.164000e+10,9.392000e+10,3.751000e+15,...,11580.0,93650.0,-179000000.0,-1.441000e+09,-1.159000e+10,6.230000e+12,1.293000e+10,1.041000e+11,3.248000e+15,'TRHFHFF128F930AC11'


Feature data corresponding to dataset name: msd-jmir-lpc-all-v1.0:


,LPC_Overall_Standard_Deviation_1,LPC_Overall_Standard_Deviation_2,LPC_Overall_Standard_Deviation_3,LPC_Overall_Standard_Deviation_4,LPC_Overall_Standard_Deviation_5,LPC_Overall_Standard_Deviation_6,LPC_Overall_Standard_Deviation_7,LPC_Overall_Standard_Deviation_8,LPC_Overall_Standard_Deviation_9,LPC_Overall_Standard_Deviation_10,...,LPC_Overall_Average_2,LPC_Overall_Average_3,LPC_Overall_Average_4,LPC_Overall_Average_5,LPC_Overall_Average_6,LPC_Overall_Average_7,LPC_Overall_Average_8,LPC_Overall_Average_9,LPC_Overall_Average_10,MSD_TRACKID
0,0.04652,0.1125,0.1386,0.18290,0.09227,0.1261,0.06775,0.11680,0.08311,0.0,...,0.6449,-0.03497,-0.17800,-0.27790,0.10960,-0.140400,0.02397,-0.12520,0.0,'TRMMMYQ128F932D901'
1,0.05153,0.1267,0.1630,0.15280,0.11270,0.1794,0.07344,0.08478,0.06295,0.0,...,0.7336,0.11510,-0.21120,-0.23690,0.14430,-0.081700,-0.01350,-0.13160,0.0,'TRMMMKD128F425225D'
2,0.04112,0.1182,0.1352,0.09433,0.14480,0.1185,0.10710,0.08826,0.12590,0.0,...,0.3939,-0.24540,-0.08293,-0.32000,0.11670,-0.087040,-0.10010,-0.14580,0.0,'TRMMMRX128F93187D9'
3,0.08796,0.1409,0.1781,0.13080,0.15690,0.1263,0.12650,0.10070,0.10500,0.0,...,0.4590,-0.25060,0.01747,-0.28360,0.21950,-0.055110,-0.02631,-0.09313,0.0,'TRMMMCH128F425532C'
4,0.02573,0.1512,0.1432,0.14310,0.10650,0.1196,0.07276,0.08836,0.05992,0.0,...,0.6478,0.09408,0.06476,-0.01578,0.06642,0.004455,-0.01168,-0.03808,0.0,'TRMMMWA128F426B589'


Feature data corresponding to dataset name: msd-jmir-methods-of-moments-all-v1.0:


...

,component_0,component_1,component_2,component_3,component_4,component_5,component_6,component_7,component_8,component_9,...,component_159,component_160,component_161,component_162,component_163,component_164,component_165,component_166,component_167,instanceName
0,3.340253,3.184129,3.467989,3.961744,3.802365,4.048197,3.617043,3.335274,3.111133,2.560841,...,9.814441,12.419846,9.361515,7.508801,6.970781,4.840366,4.327801,2.079462,0.001066,'TRJAPJV128F147E501'
1,3.859655,3.169665,2.339668,2.379815,2.087785,3.719538,3.033265,2.719684,2.258551,2.830153,...,9.991244,7.845734,8.712100,4.766188,3.607995,3.487868,2.994789,1.304786,0.001066,'TRKXUVI12903CF164C'
2,2.366181,3.693143,3.689352,3.922064,3.597496,3.675475,3.366070,3.149710,2.901223,2.544667,...,12.181783,9.146891,7.610539,6.840762,5.609022,4.445043,3.850136,2.175525,0.412306,'TRVSMHB128F933868B'
3,2.049054,4.118874,3.839635,3.426112,2.437866,2.808449,2.486959,2.366762,2.135613,1.633703,...,6.914300,7.312521,6.697511,6.286495,5.046462,3.163909,2.540429,1.371564,0.001066,'TRIIFIJ128F425D096'
4,4.407492,4.824841,4.080729,4.589847,3.908415,3.944584,4.569623,4.178122,4.202005,3.411257,...,18.883988,18.684402,12.268226,9.667586,7.543367,9.391542,11.664551,6.619618,1.100809,'TRKKTRL128F933DE30'


Feature data corresponding to dataset name: msd-trh-v1.0:


,component_0,component_1,component_2,component_3,component_4,component_5,component_6,component_7,component_8,component_9,...,component_411,component_412,component_413,component_414,component_415,component_416,component_417,component_418,component_419,instanceName
0,7.663735,6.981061,8.467507,6.571336,5.633960,8.180509,4.314876,4.100904,5.979875,3.670057,...,2.095989,1.726187,2.308420,2.044628,1.671321,2.294742,1.787176,1.507227,2.970258,'TRBRYCM128F42681FE'
1,17.145029,13.643089,10.666042,8.857338,8.446842,10.066524,6.726110,6.230044,6.250445,4.958765,...,1.682910,1.727391,1.796884,1.742186,1.518526,1.722430,1.792980,1.530533,1.903642,'TRSZPGP128F1456CE3'
2,14.535889,11.364023,8.664688,9.720170,8.196153,5.545929,6.525569,5.897498,12.678313,5.139655,...,3.553840,3.645958,5.870902,4.138360,3.004171,2.786075,3.196151,2.980204,2.810410,'TRUHNXZ128F1496B8F'
3,6.608727,5.889982,5.242445,4.541180,5.458274,3.798462,4.941622,5.361690,3.752878,3.690395,...,4.811908,3.375773,3.118121,2.358886,4.413413,2.764471,5.206478,2.746375,3.467102,'TRQODPB128F934C699'
4,9.719172,11.088511,18.813900,6.392022,7.077559,7.246826,4.931072,6.396743,6.104538,5.809380,...,2.832877,3.334148,3.453700,2.614119,2.446143,2.813223,3.107177,2.926267,2.421697,'TRMNUKS128F4230ED5'


Feature data corresponding to dataset name: msd-tssd-v1.0:


,component_1,component_2,component_3,component_4,component_5,component_6,component_7,component_8,component_9,component_10,...,component_1168,component_1169,component_1170,component_1171,component_1172,component_1173,component_1174,component_1175,component_1176,instanceName
0,3.340253,3.184129,3.467989,3.961744,3.802365,4.048197,3.617043,3.335274,3.111133,2.560841,...,11.328043,17.813001,11.465108,8.981480,7.833965,5.010287,5.289220,2.326150,0.001066,TRJAPJV128F147E501
1,3.859655,3.169665,2.339668,2.379815,2.087785,3.719538,3.033265,2.719684,2.258551,2.830153,...,13.345222,9.088343,8.852256,4.873721,3.655026,3.641957,3.029004,1.315704,0.001066,TRKXUVI12903CF164C
2,2.366181,3.693143,3.689352,3.922064,3.597496,3.675475,3.366070,3.149710,2.901223,2.544667,...,14.520047,10.333855,8.751075,9.647560,7.675982,5.672750,5.248331,2.370116,0.444731,TRVSMHB128F933868B
3,2.049054,4.118874,3.839635,3.426112,2.437866,2.808449,2.486959,2.366762,2.135613,1.633703,...,8.493671,8.592187,8.018544,9.971325,6.184466,4.542195,4.012239,2.229550,0.001066,TRIIFIJ128F425D096
4,4.407492,4.824841,4.080729,4.589847,3.908415,3.944584,4.569623,4.178122,4.202005,3.411257,...,22.558385,19.272145,13.098851,11.940146,8.060847,10.481845,15.760706,7.519517,1.205712,TRKKTRL128F933DE30


### C. Do you think these audio feature attribute names are convenient to use as column names?

### C.1 Develop a systematic way to rename columns in the audio feature datasets after you load them in Spark 

In [24]:
# List of dataset names
dataset_names = [
    'msd-jmir-area-of-moments-all-v1.0',
    'msd-jmir-lpc-all-v1.0', 
    'msd-jmir-methods-of-moments-all-v1.0', 
    'msd-jmir-mfcc-all-v1.0',
    'msd-jmir-spectral-all-all-v1.0',
    'msd-jmir-spectral-derivatives-all-all-v1.0',
    'msd-marsyas-timbral-v1.0',
    'msd-mvd-v1.0',
    'msd-rh-v1.0',
    'msd-rp-v1.0',
    'msd-ssd-v1.0',
    'msd-trh-v1.0',
    'msd-tssd-v1.0',   
]

# Mapping dataset names to their corresponding prefixes
dataset_prefix_mapping = {
    'msd-jmir-area-of-moments-all-v1.0': 'AMM_',
    'msd-jmir-lpc-all-v1.0': 'LPC_',
    'msd-jmir-methods-of-moments-all-v1.0': 'MoM_',
    'msd-jmir-mfcc-all-v1.0': 'MFCC_',
    'msd-jmir-spectral-all-all-v1.0': 'Spectral_All_',
    'msd-jmir-spectral-derivatives-all-all-v1.0': 'Spectral_Deri_',
    'msd-marsyas-timbral-v1.0': 'timbral_',
    # The following datasets do not require renaming
    'msd-mvd-v1.0': None,
    'msd-rh-v1.0': None,
    'msd-rp-v1.0': None,
    'msd-ssd-v1.0': None,
    'msd-trh-v1.0': None,
    'msd-tssd-v1.0': None,
}

# Loop through each dataset name
for dataset_name in dataset_names:
    # Set paths dynamically for the attribute and feature files
    attribute_path = f'hdfs:///data/msd/audio/attributes/{dataset_name}.attributes.csv'
    feature_path = f'hdfs:///data/msd/audio/features/{dataset_name}.csv'
    
    # Load attribute data to create the schema
    attribute_data = (spark.read.format("csv")
                      .option("header", False)
                      .load(attribute_path)
                      .toDF("name", "type"))
    
    # Create schema using the attribute data (assuming generate_schema is defined elsewhere)
    feature_schema = generate_schema(attribute_data)
    
    # Load feature data using the generated schema
    feature_data = (spark.read.format("csv")
                    .option("header", False)
                    .option("inferSchema", False)
                    .schema(feature_schema)
                    .load(feature_path))
    
    # Rename columns based on the mapping
    prefix = dataset_prefix_mapping.get(dataset_name, None)
    if prefix:
        # Get the old column names
        old_columns = feature_data.columns
        new_columns = []

        # Iterate over the columns and apply renaming logic, skipping any column that starts with 'MSD'
        for i, old_col in enumerate(old_columns):
            if old_col.startswith('MSD') or old_col.startswith('track_id'):
                new_columns.append(old_col)  # Keep MSD columns unchanged
            else:
                if i < 10:
                    # First 10 columns: std pattern
                    new_columns.append(f"{prefix}std_{i + 1}")
                elif i < 20:
                    # Next 10 columns: avg pattern
                    new_columns.append(f"{prefix}avg_{i - 9}")
                else:
                    # Remaining columns (if more than 20): sequential pattern
                    new_columns.append(f"{prefix}{i + 1}")
        
        # Rename the columns using withColumnRenamed
        for old_col, new_col in zip(old_columns, new_columns):
            feature_data = feature_data.withColumnRenamed(old_col, new_col)
    
    # Show the result for each dataset
    print(f"Feature data corresponding to dataset name: {dataset_name}:")
    # feature_data.printSchema()
    show_as_html(feature_data, 5)

Feature data corresponding to dataset name: msd-jmir-area-of-moments-all-v1.0:


,AMM_std_1,AMM_std_2,AMM_std_3,AMM_std_4,AMM_std_5,AMM_std_6,AMM_std_7,AMM_std_8,AMM_std_9,AMM_std_10,...,AMM_avg_2,AMM_avg_3,AMM_avg_4,AMM_avg_5,AMM_avg_6,AMM_avg_7,AMM_avg_8,AMM_avg_9,AMM_avg_10,MSD_TRACKID
0,1.2000,3355.0,26270.0,39850000.0,3.096000e+08,2.403000e+09,8.749000e+11,2.775000e+09,2.155000e+10,4.064000e+14,...,5746.0,43470.0,-44220000.0,-3.376000e+08,-2.576000e+09,7.665000e+11,3.015000e+09,2.302000e+10,3.460000e+14,'TRHFHQZ12903C9E2D5'
1,0.9295,6720.0,44100.0,160800000.0,1.060000e+09,6.985000e+09,7.095000e+12,9.545000e+09,6.293000e+10,2.037000e+15,...,11580.0,74040.0,-179200000.0,-1.153000e+09,-7.420000e+09,6.242000e+12,1.037000e+10,6.680000e+10,1.694000e+15,'TRHFHYX12903CAF953'
2,1.8830,6712.0,49060.0,160600000.0,1.176000e+09,8.609000e+09,7.083000e+12,1.058000e+10,7.744000e+10,2.781000e+15,...,11580.0,85200.0,-179100000.0,-1.316000e+09,-9.660000e+09,6.233000e+12,1.182000e+10,8.680000e+10,2.463000e+15,'TRHFHAU128F9341A0E'
3,1.8840,6722.0,56130.0,161000000.0,1.346000e+09,1.127000e+10,7.112000e+12,1.211000e+10,1.014000e+11,4.193000e+15,...,11600.0,93320.0,-179700000.0,-1.459000e+09,-1.185000e+10,6.262000e+12,1.311000e+10,1.066000e+11,3.432000e+15,'TRHFHLP128F14947A7'
4,1.5200,6709.0,53230.0,160500000.0,1.295000e+09,1.045000e+10,7.076000e+12,1.164000e+10,9.392000e+10,3.751000e+15,...,11580.0,93650.0,-179000000.0,-1.441000e+09,-1.159000e+10,6.230000e+12,1.293000e+10,1.041000e+11,3.248000e+15,'TRHFHFF128F930AC11'


Feature data corresponding to dataset name: msd-jmir-lpc-all-v1.0:


,LPC_std_1,LPC_std_2,LPC_std_3,LPC_std_4,LPC_std_5,LPC_std_6,LPC_std_7,LPC_std_8,LPC_std_9,LPC_std_10,...,LPC_avg_2,LPC_avg_3,LPC_avg_4,LPC_avg_5,LPC_avg_6,LPC_avg_7,LPC_avg_8,LPC_avg_9,LPC_avg_10,MSD_TRACKID
0,0.04652,0.1125,0.1386,0.18290,0.09227,0.1261,0.06775,0.11680,0.08311,0.0,...,0.6449,-0.03497,-0.17800,-0.27790,0.10960,-0.140400,0.02397,-0.12520,0.0,'TRMMMYQ128F932D901'
1,0.05153,0.1267,0.1630,0.15280,0.11270,0.1794,0.07344,0.08478,0.06295,0.0,...,0.7336,0.11510,-0.21120,-0.23690,0.14430,-0.081700,-0.01350,-0.13160,0.0,'TRMMMKD128F425225D'
2,0.04112,0.1182,0.1352,0.09433,0.14480,0.1185,0.10710,0.08826,0.12590,0.0,...,0.3939,-0.24540,-0.08293,-0.32000,0.11670,-0.087040,-0.10010,-0.14580,0.0,'TRMMMRX128F93187D9'
3,0.08796,0.1409,0.1781,0.13080,0.15690,0.1263,0.12650,0.10070,0.10500,0.0,...,0.4590,-0.25060,0.01747,-0.28360,0.21950,-0.055110,-0.02631,-0.09313,0.0,'TRMMMCH128F425532C'
4,0.02573,0.1512,0.1432,0.14310,0.10650,0.1196,0.07276,0.08836,0.05992,0.0,...,0.6478,0.09408,0.06476,-0.01578,0.06642,0.004455,-0.01168,-0.03808,0.0,'TRMMMWA128F426B589'


Feature data corresponding to dataset name: msd-jmir-methods-of-moments-all-v1.0:


...

,component_0,component_1,component_2,component_3,component_4,component_5,component_6,component_7,component_8,component_9,...,component_159,component_160,component_161,component_162,component_163,component_164,component_165,component_166,component_167,instanceName
0,3.340253,3.184129,3.467989,3.961744,3.802365,4.048197,3.617043,3.335274,3.111133,2.560841,...,9.814441,12.419846,9.361515,7.508801,6.970781,4.840366,4.327801,2.079462,0.001066,'TRJAPJV128F147E501'
1,3.859655,3.169665,2.339668,2.379815,2.087785,3.719538,3.033265,2.719684,2.258551,2.830153,...,9.991244,7.845734,8.712100,4.766188,3.607995,3.487868,2.994789,1.304786,0.001066,'TRKXUVI12903CF164C'
2,2.366181,3.693143,3.689352,3.922064,3.597496,3.675475,3.366070,3.149710,2.901223,2.544667,...,12.181783,9.146891,7.610539,6.840762,5.609022,4.445043,3.850136,2.175525,0.412306,'TRVSMHB128F933868B'
3,2.049054,4.118874,3.839635,3.426112,2.437866,2.808449,2.486959,2.366762,2.135613,1.633703,...,6.914300,7.312521,6.697511,6.286495,5.046462,3.163909,2.540429,1.371564,0.001066,'TRIIFIJ128F425D096'
4,4.407492,4.824841,4.080729,4.589847,3.908415,3.944584,4.569623,4.178122,4.202005,3.411257,...,18.883988,18.684402,12.268226,9.667586,7.543367,9.391542,11.664551,6.619618,1.100809,'TRKKTRL128F933DE30'


Feature data corresponding to dataset name: msd-trh-v1.0:


,component_0,component_1,component_2,component_3,component_4,component_5,component_6,component_7,component_8,component_9,...,component_411,component_412,component_413,component_414,component_415,component_416,component_417,component_418,component_419,instanceName
0,7.663735,6.981061,8.467507,6.571336,5.633960,8.180509,4.314876,4.100904,5.979875,3.670057,...,2.095989,1.726187,2.308420,2.044628,1.671321,2.294742,1.787176,1.507227,2.970258,'TRBRYCM128F42681FE'
1,17.145029,13.643089,10.666042,8.857338,8.446842,10.066524,6.726110,6.230044,6.250445,4.958765,...,1.682910,1.727391,1.796884,1.742186,1.518526,1.722430,1.792980,1.530533,1.903642,'TRSZPGP128F1456CE3'
2,14.535889,11.364023,8.664688,9.720170,8.196153,5.545929,6.525569,5.897498,12.678313,5.139655,...,3.553840,3.645958,5.870902,4.138360,3.004171,2.786075,3.196151,2.980204,2.810410,'TRUHNXZ128F1496B8F'
3,6.608727,5.889982,5.242445,4.541180,5.458274,3.798462,4.941622,5.361690,3.752878,3.690395,...,4.811908,3.375773,3.118121,2.358886,4.413413,2.764471,5.206478,2.746375,3.467102,'TRQODPB128F934C699'
4,9.719172,11.088511,18.813900,6.392022,7.077559,7.246826,4.931072,6.396743,6.104538,5.809380,...,2.832877,3.334148,3.453700,2.614119,2.446143,2.813223,3.107177,2.926267,2.421697,'TRMNUKS128F4230ED5'


Feature data corresponding to dataset name: msd-tssd-v1.0:


,component_1,component_2,component_3,component_4,component_5,component_6,component_7,component_8,component_9,component_10,...,component_1168,component_1169,component_1170,component_1171,component_1172,component_1173,component_1174,component_1175,component_1176,instanceName
0,3.340253,3.184129,3.467989,3.961744,3.802365,4.048197,3.617043,3.335274,3.111133,2.560841,...,11.328043,17.813001,11.465108,8.981480,7.833965,5.010287,5.289220,2.326150,0.001066,TRJAPJV128F147E501
1,3.859655,3.169665,2.339668,2.379815,2.087785,3.719538,3.033265,2.719684,2.258551,2.830153,...,13.345222,9.088343,8.852256,4.873721,3.655026,3.641957,3.029004,1.315704,0.001066,TRKXUVI12903CF164C
2,2.366181,3.693143,3.689352,3.922064,3.597496,3.675475,3.366070,3.149710,2.901223,2.544667,...,14.520047,10.333855,8.751075,9.647560,7.675982,5.672750,5.248331,2.370116,0.444731,TRVSMHB128F933868B
3,2.049054,4.118874,3.839635,3.426112,2.437866,2.808449,2.486959,2.366762,2.135613,1.633703,...,8.493671,8.592187,8.018544,9.971325,6.184466,4.542195,4.012239,2.229550,0.001066,TRIIFIJ128F425D096
4,4.407492,4.824841,4.080729,4.589847,3.908415,3.944584,4.569623,4.178122,4.202005,3.411257,...,22.558385,19.272145,13.098851,11.940146,8.060847,10.481845,15.760706,7.519517,1.205712,TRKKTRL128F933DE30


In [25]:
# Run this cell before closing the notebook or kill your spark application by hand using the link in the Spark UI

stop_spark()